In [1]:
import torch
import torchphysics as tp
import pytorch_lightning as pl
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

/home/nick7/anaconda3/envs/bosch-pde/lib/python3.9/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


In [2]:
# Spaces 
T = tp.spaces.R1('t') # input variable
U = tp.spaces.R1('u') # output variable
K = tp.spaces.R1('k') # parameter
F = tp.spaces.R1('f') # function spaces name
# Domains
T_int = tp.domains.Interval(T, 0, 1)
K_int = tp.domains.Interval(K, 0, 6)

In [3]:
# Defining function set
Fn_space = tp.spaces.FunctionSpace(T_int, F)

def f1(k, t):
    return k*t

def f2(k, t):
    return k*t**2

def f3(k, t):
    return k*torch.cos(k*t)

param_sampler = tp.samplers.RandomUniformSampler(K_int, n_points=20)
Fn_set = tp.domains.CustomFunctionSet(Fn_space, param_sampler, f1)
Fn_set += tp.domains.CustomFunctionSet(Fn_space, param_sampler, f2)
Fn_set += tp.domains.CustomFunctionSet(Fn_space, param_sampler, f3)

In [4]:
# Model
dis_sampler = tp.samplers.GridSampler(T_int, 50).make_static()
trunk_net = tp.models.FCTrunkNet(T, U, output_neurons=50)
branch_net = tp.models.FCBranchNet(Fn_space, U, output_neurons=50, discretization_sampler=dis_sampler)
model = tp.models.DeepONet(trunk_net, branch_net)

In [5]:
# ODE condition
inner_sampler = tp.samplers.RandomUniformSampler(T_int, 500)

def ode_residual(u, t, f):
    return tp.utils.grad(u, t) - f

ode_cond = tp.conditions.PIDeepONetCondition(deeponet_model=model, 
                                             function_set=Fn_set, 
                                             output_sampler=inner_sampler, 
                                             name='ode_condition',
                                             residual_fn=ode_residual)

In [6]:
left_sampler = tp.samplers.RandomUniformSampler(T_int.boundary_left, 500)

def initial_residual(u):
    return u

initial_cond = tp.conditions.PIDeepONetCondition(deeponet_model=model, 
                                                 function_set=Fn_set, 
                                                 output_sampler=left_sampler, 
                                                 residual_fn=initial_residual)

In [7]:
solver = tp.solver.Solver([ode_cond, initial_cond])

trainer = pl.Trainer(gpus='-1' if torch.cuda.is_available() else None,
                     num_sanity_val_steps=0,
                     benchmark=True,
                     max_steps=3000,
                     logger=False,
                     checkpoint_callback=False
                     )

trainer.fit(solver)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]

  | Name             | Type       | Params
------------------------------------------------
0 | train_conditions | ModuleList | 4.8 K 
1 | val_conditions   | ModuleList | 0     
------------------------------------------------
4.8 K     Trainable params
0         Non-trainable params
4.8 K     Total params
0.019     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]